(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Clustering the data from the Mayor's 24-hour hotline in Boston

## Introduction 

In this assignment, we will do a clustering analysis on the 311 calls that we used in the previous homework. We will focus on choosing a value for k, clustering based on multiple features and detecting outliers.

---------------------------

In [36]:
%matplotlib inline
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
boston = pd.read_csv('../Homework-2/cases_2015.csv')

## Step 1. Simple clustering

The first thing that we will try is to cluster the geolocation data of the calls. The geolocation of a record consists of a pair `(latitude, longitude)`, which represents the position of the record on the map. Since the dataset that we have spans only a small geographic location, i.e. Boston, we will use the Euclidean distance when measuring how close two points are. Techinically, we should pick a [projection](https://github.com/mbostock/d3/wiki/Geo-Projections) in order to transform the data into 2d, and measure  the Euclidean distance in the projected space.

Since we are dealing with numeric data, we might consider centering and normalizing them. In order to do that, check the [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) from `scikit-learn`.

For exampe, the result of a clustering with 10 clusters is the following:


![A clustering of the locations of the calls for k=10](clusters10.png)

Filter the dataset to get the latitude and longitude, transform it as required and run the k-means clustering algorithm. Argue about the number of clusters you used. **(pts 50)**

In [ ]:
import numpy as np
import scipy as sp
import sklearn.datasets as sd
import sklearn.metrics as sm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler as std

data = std().fit_transform(np.array(boston[['LATITUDE','LONGITUDE']]))

# Find the most suitable K value
def chooseKcluster(data,K):
    count = 0
    for k in range(1,K+1):
        kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
        error = kmeans.fit_predict(data)
        error = kmeans.inertia_
        count = count + 1
        print (count, error)
        
# run the following to see the error rate regarding to k values
# we might see the error rate reaches convergence when k = 20
# run the findK() takes a couple of minutes when k is set to large number
# comment the following line to save time
chooseKcluster(data,30)




# the code below is from piazza page, thanks to Harry
fig, ax = plt.subplots(1,1,figsize=(10,10))
cmap = sns.color_palette("bright", n_colors=4)
cmap.extend(sns.hls_palette(n_colors=16, h=0.7, l=0.4, s=0.8))
X = std().fit_transform(np.array(boston['LONGITUDE']))
Y = std().fit_transform(np.array(boston['LATITUDE']))

mean = KMeans(init='k-means++', n_clusters=20, n_init=20)
mean.fit_predict(data)
lbs = mean.labels_

for i, label in enumerate(set(lbs)):
    point_indices = np.where(lbs == label)[0]
    point_indices = point_indices.tolist()
    plt.scatter(X[point_indices], Y[point_indices], s=25, alpha=0.2, c=cmap[i], marker='+')

(1, 93395.999999997468)
(2, 48238.857418424239)
(3, 32112.654052525042)
(4, 21165.742901245794)
(5, 16350.889180200842)
(6, 13547.981118382579)
(7, 11527.778745559643)
(8, 9567.8523987705448)
(9, 7814.739332014864)
(10, 6836.1856748446335)
(11, 5932.5954086723605)
(12, 5275.6210678272582)
(13, 4886.8771438041113)
(14, 4483.5360888715923)

-----

## Step 2. Using multiple features

The next step in our analysis is to use more features than just the geolocation of a record (whose clustering only gives us information about the neighborhoods, which should not be a huge revelation!) Filter the dataset to get more features, transform them as required and compute another clustering. You should argue about the quality of your clustering and use domain knowledge (i.e. the fact that you are locals) to explore the clusters you produced and present them to the reader. **(pts 30)**

In [35]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


new_Boston = boston[['fire_district','city_council_district','LOCATION_ZIPCODE','Property_ID']]
new_Boston = Imputer(missing_values='NaN').fit_transform(new_Boston)

new_Boston = StandardScaler().fit_transform(new_Boston)


enc = LabelEncoder()
uniq = pd.Series(boston['CASE_TITLE'].values.ravel()).unique()
enc.fit(uniq)
list(enc.classes_)
new_boston1 = enc.transform(boston['CASE_TITLE'].values)

Alist = new_Boston.tolist()
Applist = new_boston1.tolist()

l = len(new_boston1)
for i in range(l):
    Alist[i].append(Applist[i])
new_training = np.array(Alist)

chooseKcluster(new_training,20)
    
print "W should choose K = 10 since the error rate decreases to convergence when K>10"

(1, 1045595023.9832969)
(2, 252791836.51044863)
(3, 88761696.969217554)
(4, 42476234.125261307)
(5, 19900729.737081759)
(6, 15128885.625619439)
(7, 12007846.022703381)
(8, 8867798.474031752)
(9, 6416158.7579004234)
(10, 4884323.6237996928)
(11, 3741294.9071201705)
(12, 3190408.9152947599)
(13, 2790733.2703276197)
(14, 2427180.120352638)
(15, 2085331.7966803517)
(16, 1761025.8369400105)
(17, 1525497.4402606017)
(18, 1352702.0958752634)
(19, 1159070.2074704973)
(20, 1046187.7963990686)
W should choose K = 10 since the error rate decreases to convergence when K>10


-----

## Step 3. Detecting outliers

Now that we have a clustering of the records in the dataset, we can detect the outlying ones. These are the ones who are the farthest from the centroids of their clusters. Track them down and describe any interesting observations that you can make. **(pts 20)** 

In [ ]:
# Calculate the Euclidean distance between 2 given points

leng = len(centroids)

outliers = range(leng)
for k in outliers:
    centers = centroids[k]
    maxDist = 0
    for i in range(l):
        if labels[i] == k:
            dist = np.sqrt(np.sum((new_training[i]-centers)**2))
            if dist > maxDist:
                maxDist = dist
                outliers[k] = new_training[i]
outliers

In [34]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()